In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [3]:
df=pd.read_csv(r'D:\DOE實驗設計\1.csv',encoding='ANSI')

In [4]:
m=lambda x:min(df[x])
n=lambda x:max(df[x])
bounds = [(m('par1'),n('par1')),(m('par2'),n('par2')),(m('par3'),n('par3')),(m('par4'),n('par4')),(m('par5'),n('par5')),(m('par6'),n('par6'))]
bounds

[(64.0, 512.0),
 (32.0, 256.0),
 (8.0, 64.0),
 (4.0, 16.0),
 (0.0001, 0.01),
 (0.01, 0.1)]

In [40]:
df.iloc[:,0:4]=np.log2(df.iloc[:,0:4])
df.iloc[:,4:6]=np.log10(df.iloc[:,4:6])

In [5]:
#normalization
p=preprocessing.MinMaxScaler()
df.iloc[:,:6]=p.fit_transform(df.iloc[:,:6])

In [42]:
df.head()

,par1,par2,par3,par4,par5,par6,train_accuracy,test_accuracy,sen,speci,NPV,PPV,Fmeasure,TN,FP,TP,FN
0,0.333333,0.333333,0.333333,0.0,0.682217,0.682217,94.169406,91.732136,46.250,94.627935,96.509740,35.406699,40.108401,2378,135,74,86
1,0.666667,0.333333,0.333333,0.0,0.682217,0.682217,94.409066,92.293303,44.375,95.344210,96.418511,37.765957,40.804598,2396,117,71,89
2,0.333333,0.666667,0.333333,0.0,0.682217,0.682217,93.696932,90.497568,44.375,93.434142,96.347969,30.084746,35.858586,2348,165,71,89
3,0.666667,0.666667,0.333333,0.0,0.682217,0.682217,94.617913,91.283202,38.125,94.667728,96.004843,31.282051,34.366197,2379,134,61,99
4,0.333333,0.333333,0.666667,0.0,0.682217,0.682217,93.909203,92.218481,37.500,95.702348,96.007984,35.714286,36.585366,2405,108,60,100


In [6]:
# bounds = [(m('par1'),n('par1')),(m('par2'),n('par2')),(m('par3'),n('par3')),(m('par4'),n('par4')),(m('par5'),n('par5')),(m('par6'),n('par6'))]
bounds = [(0,1) for i in range(6)]
bounds

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

In [7]:
#fit
from sklearn.linear_model import LinearRegression
X=df.iloc[:,:6]
y1=df['PPV']
y2=df['Fmeasure']

from sklearn.preprocessing import PolynomialFeatures 

poly = PolynomialFeatures(degree = 1)
X_poly = poly.fit_transform(X)

def polyfit(x):
    l1=LinearRegression().fit(x, y1)
    l2=LinearRegression().fit(x, y2)
#     l1=ElasticNet(alpha=0.01,l1_ratio=1).fit(x, y1)
#     l2=ElasticNet(alpha=0.01,l1_ratio=1).fit(x, y2)
    print('y1_score:%f  y2_score:%f'%(l1.score(x, y1),l2.score(x, y2)))
    return l1,l2
polyfit(X)
l1,l2=polyfit(X_poly)

def eggholder(x):
#     a=0
#     for i,j in enumerate([1,x[0],x[1],x[2],x[0]**2,x[1]**2,x[2]**2,x[0]*x[1],x[2]*x[1],x[0]*x[2]]):
#         a+=l1coef(i)*j
#         b=l2coef(0)+x[0]*l2coef(1)+x[1]*l2coef(2)+x[0]*x[0]*l2coef(3)+x[0]*x[1]*l2coef(4)+x[1]*x[1]*l2coef(5)
#         c=np.sqrt(a*b)
    a=(l1.predict(poly.fit_transform([[x[0],x[1],x[2],x[3],x[4],x[5]]])))[0]
    b=(l2.predict(poly.fit_transform([[x[0],x[1],x[2],x[3],x[4],x[5]]])))[0]
#     a=1
    a=np.where(a>0,a,0);b=np.where(b>0,b,0);
#     a=np.where(a>0,np.where(a>100,100,a),0);b=np.where(b>0,np.where(b>100,100,b),0);
    c=np.sqrt(a*b)
    return -c

#opt
from scipy import optimize
results = dict()

#     from numpy.random import random
#     r = random(6)  # Random start vector
#     res = minimize(lambda x: np.var(x), r, bounds)

results['min']= optimize.minimize(eggholder, bounds)
results['min']['fun']=0
# results['mini'] = optimize.minimize(eggholder, bounds)
results['shgo'] = optimize.shgo(eggholder, bounds)
results['DA'] = optimize.dual_annealing(eggholder, bounds)
results['DE'] = optimize.differential_evolution(eggholder, bounds)
# results['BH'] = optimize.basinhopping(eggholder, bounds)

for key in results:
    if results['min']['fun']>results[key]['fun']:
        results['min']=results[key]
        print(key)

print('Optimize:')
print(results['min'])


print('\nOptimize Reselt:')
app=lambda x:np.append(results[x]['x'][0:],results[x]['fun'])
display(pd.DataFrame([app('shgo'),app('DA'),app('DE')]
                     ,columns=['X1','X2','X3','X4','X5','X6','fun'],index=['shgo','DA','DE']))
display(pd.DataFrame([p.inverse_transform([results['shgo']['x']])[0],p.inverse_transform([results['DA']['x']])[0],p.inverse_transform([results['DE']['x']])[0]]
                     ,columns=['X1','X2','X3','X4','X5','X6'],index=['shgo','DA','DE']))

y1_score:0.165422  y2_score:0.074919
y1_score:0.165422  y2_score:0.074919
shgo
Optimize:
     fun: -37.49668343412393
    funl: array([-37.49668343])
 message: 'Optimization terminated successfully.'
    nfev: 73
     nit: 2
   nlfev: 8
   nlhev: 0
   nljev: 1
 success: True
       x: array([1., 0., 0., 1., 1., 0.])
      xl: array([[1., 0., 0., 1., 1., 0.]])

Optimize Reselt:


,X1,X2,X3,X4,X5,X6,fun
shgo,1.0,0.0,0.0,1.0,1.0,0.0,-37.496683
DA,1.0,0.0,0.0,1.0,1.0,0.0,-37.496683
DE,1.0,0.0,0.0,1.0,1.0,0.0,-37.496683


,X1,X2,X3,X4,X5,X6
shgo,512.0,32.0,8.0,16.0,0.01,0.01
DA,512.0,32.0,8.0,16.0,0.01,0.01
DE,512.0,32.0,8.0,16.0,0.01,0.01


In [76]:
from scipy.optimize import linprog
C = [1,-4] 
# A = [[-3,1],[1,2]]
# b = [6,4]
X0_bounds = [5,10]
X1_bounds = [-3,10]
res = linprog(C,bounds=(X0_bounds,X1_bounds))
print(res)

     con: array([], dtype=float64)
     fun: -35.0
 message: 'The solution was determined in presolve as there are no non-trivial constraints.'
     nit: 0
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 5., 10.])


In [ ]:
from scipy.optimize import linprog
C = l1.coef_

res = linprog(C,bounds)
print(res)

In [43]:
results['shgo']

     fun: -18718954.823032275
    funl: array([-18718954.82303227])
 message: 'Optimization terminated successfully.'
    nfev: 81
     nit: 2
   nlfev: 16
   nlhev: 0
   nljev: 2
 success: True
       x: array([0. , 1. , 0. , 0.5, 0. , 0. ])
      xl: array([[0. , 1. , 0. , 0.5, 0. , 0. ]])

In [5]:
import pyDOE2
ccd=pyDOE2.ccdesign(6,center=(6,4),face='cci')
ccd=preprocessing.MinMaxScaler().fit_transform(ccd)
# pd.DataFrame(ccd,columns=df.columns[:6])

In [36]:
pd.DataFrame(ccd,columns=df.columns[:6]).to_csv('D:\DOE實驗設計\最佳化組合.csv',index=0)

In [8]:
from sklearn.linear_model import ElasticNet
poly = PolynomialFeatures(degree = 3)
X_poly = poly.fit_transform(X)
l1=LinearRegression().fit(X_poly, y1)
l2=LinearRegression().fit(X_poly, y2)
l3=ElasticNet(alpha=0.01,l1_ratio=1).fit(X_poly, y1)
l4=ElasticNet(alpha=0.01,l1_ratio=1).fit(X_poly, y2)
print('y1_score:%f  y2_score:%f'%(l1.score(X_poly, y1),l2.score(X_poly, y2)))
print('y1_score:%f  y2_score:%f'%(l3.score(X_poly, y1),l4.score(X_poly, y2)))

y1_score:0.619237  y2_score:0.637415
y1_score:0.402135  y2_score:0.532964


In [17]:
m=lambda x:min(df[x])
n=lambda x:max(df[x])
bounds = [(m('par1'),n('par1')),(m('par2'),n('par2')),(m('par3'),n('par3')),(m('par4'),n('par4')),(m('par5'),n('par5')),(m('par6'),n('par6'))]
bounds

[(64.0, 512.0),
 (32.0, 256.0),
 (8.0, 64.0),
 (4.0, 16.0),
 (0.0001, 0.01),
 (0.01, 0.1)]

In [33]:
ccd=pyDOE2.ccdesign(6,center=(4,4),face='cci')
ccd=preprocessing.MinMaxScaler().fit_transform(ccd)
pd.DataFrame(ccd,columns=df.columns[:6])

,par1,par2,par3,par4,par5,par6
0,0.317783,0.317783,0.317783,0.317783,0.317783,0.317783
1,0.682217,0.317783,0.317783,0.317783,0.317783,0.317783
2,0.317783,0.682217,0.317783,0.317783,0.317783,0.317783
3,0.682217,0.682217,0.317783,0.317783,0.317783,0.317783
4,0.317783,0.317783,0.682217,0.317783,0.317783,0.317783
5,0.682217,0.317783,0.682217,0.317783,0.317783,0.317783
6,0.317783,0.682217,0.682217,0.317783,0.317783,0.317783
7,0.682217,0.682217,0.682217,0.317783,0.317783,0.317783
8,0.317783,0.317783,0.317783,0.682217,0.317783,0.317783
9,0.682217,0.317783,0.317783,0.682217,0.317783,0.317783


In [35]:
ccd=pyDOE2.ccdesign(6,center=(4,4),face='ccf')
ccd=preprocessing.MinMaxScaler().fit_transform(ccd)
pd.DataFrame(ccd,columns=df.columns[:6])

,par1,par2,par3,par4,par5,par6
0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0
5,1.0,0.0,1.0,0.0,0.0,0.0
6,0.0,1.0,1.0,0.0,0.0,0.0
7,1.0,1.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,1.0,0.0,0.0


In [81]:
d=p.inverse_transform([results['shgo']['x']])[0]
d[0:4]=2**np.round(d[0:4])
d[4:6]=10**np.round(d[4:6])

In [82]:
pd.DataFrame(d)

,0
0,512.0000
1,32.0000
2,64.0000
3,16.0000
4,0.0001
5,0.0100
